### Note:
Script 100 creates the initial daily table. This script takes successive Mediatel datasets and then appends daily rows to the initial daily table. This way we don't have to deal with the massive stacked Mediatel dataset each time we want to update the daily metrics.

Steps:
1. Load in Mediatel data each day
2. Create metric columns
3. Summarise
4. Load current summary table
5. Append to current summary table
6. Save over current summary table

In [0]:
%run ./my_packages

In [0]:
# By default mediatel dates is a list containing a single date, however, use below if need to catch up on multiple days at a time
mediatel_dates = [mediatel_date]

In [0]:
%skip
# Set `mediatel_date` here if you want to run manually. Otherwise under BAU conditions (i.e. when you don't need to run any past dates) this cell should be skipped via %skip tag.
start = '20260218'
end = '20260219'
mediatel_dates = [
    (datetime.strptime(start, "%Y%m%d") + timedelta(days=i)).strftime("%Y%m%d") for i in range((datetime.strptime(end, "%Y%m%d") - datetime.strptime(start, "%Y%m%d")).days + 1)
    ]

In [0]:
for mediatel_date in mediatel_dates:
    try:
        # Read excel file
        df = f_LoadLatestMediatel(mediatel_date=mediatel_date, select_cols=select_cols)

        # Create metrics
        df = f_CreateMetrics(df, timeframe="daily")

        # Summarise
        df_summary = f_CreateSummary(df, "daily")

        df_summary = (
            df_summary
            .with_columns(pl.col("call_date").dt.date().alias("call_date"))
            .with_columns(pl.col("call_date").dt.truncate("1w").dt.date().alias("call_week"))
            .with_columns(pl.col("call_date").dt.month_start().dt.date().alias("call_month"))
            .with_columns(pl.when(pl.col("DPD_type") == "Pre-Due").then(7).otherwise(8).alias("HC_Spin_Target"))
            .with_columns(pl.lit(5).alias("VB_Spin_Target"))
        )

        df_summary = df_summary.to_pandas()

        # Load current summary table, which will be as of t-2
        table_name = "teams.cashalo.brett_collections_funnel_daily_v2"
        df_current = spark.sql(f'SELECT * FROM {table_name}').toPandas()

        # Save it right back down as a back-up copy
        spark.sql(f"DROP TABLE IF EXISTS {table_name}_backup")
        spark_df_backup = spark.createDataFrame(df_current)
        spark_df_backup.write.saveAsTable(f"{table_name}_backup")

        # Append t-1 summary table to current t-2 summary table
        df_combined = pd.concat([df_current, df_summary], ignore_index=True)

        # Save over current t-2 summary table with new summary table which will be latest as of t-1
        spark_df = f_DailyCreateSparkTable(df_combined, table_name=table_name)
        spark_df.write.saveAsTable(table_name)

        display(spark_df)
            
    except Exception:
        dbutils.notebook.exit("Error encountered. Aborting script to prevent overwrite of summary data.")

In [0]:
%sql
SELECT * FROM teams.cashalo.brett_collections_funnel_daily_v2